# WebBaseLoader

`WebBaseLoader` 는 웹 기반 문서를 로드하는 로더입니다. 

`bs4` 라이브러리를 사용하여 웹 페이지를 파싱합니다. 

- `bs4.SoupStrainer` 를 사용하여 파싱할 요소를 지정합니다. 
- `bs_kwargs` 매개변수를 사용하여 `bs4.SoupStrainer` 의 추가적인 인수를 지정합니다. 

**참고**

- [API 도큐먼트](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html)

In [3]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 뉴스기사 내용을 로드합니다.
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000378416",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

SSL 인증 오류를 우회하기 위해, `"verify"` 옵션을 설정할 수 있습니다.

In [4]:
# ssl 인증 우회
loader.requests_kwargs = {"verify": False}

# 데이터 로드
docs = loader.load()

여러 웹페이지를 한 번에 로드할 수도 있습니다. 이를 위해 **urls**의 리스트를 로더에 전달하면, 전달된 **urls**의 순서대로 문서 리스트를 반환합니다.

In [10]:
loader = WebBaseLoader(
    web_paths=[
        "https://n.news.naver.com/article/437/0000378416",
        "https://n.news.naver.com/mnews/hotissue/article/092/0002340014?type=series&cid=2000063",
    ],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},
        )
    ),
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36",
    },
)

# 데이터 로드
docs = loader.load()

# 문서 수 확인
print(len(docs))

2


웹에서 가져온 결과를 출력합니다.

In [15]:
print(docs[0].page_content[:500])
print("===" * 10)
print(docs[1].page_content[:500])


출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책


[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨

고속 성장하는 스타트업엔 레드팀이 필요하다


[이균성의 溫技] 초심, 본질을 잃을 때한 스타트업 창업자와 최근 점심을 같이 했다. 조언을 구할 게 있다고 했다. 당장 급한 현안이 있는 건 아니었다. 여러 번 창업한 경험이 있는데 지금 하고 있는 아이템은 대박 느낌이 든다고 헸다. 그런데 오히려 더 조심해야겠다는 생각이 들더란다. 조언을 구하고자 하는 바도 성장이 예상될 때 무엇을 경계해야 할지 알고 싶다는 거였다. 적잖은 스타트업 창업자를 만났지만 드문 사례였다.2년 가까이 스타트업 창업자를 릴레이 인터뷰 하면서 의미 있게 생각했던 것이 두 가지 있다. 첫째, 회사라는 단어보다 팀이라는 어휘를 주로 쓰고 있다는 점이었다. 그 표현의 유래나 의미 때문이라기보다는 팀이라는 말이 더 정겨워 뜻 깊게 생각된 듯하다. 이해관계보다 지향하는 뜻에 더 중점을 두고 하나의 마음으로 한 곳을 향해 달려가는 집단을 가리키는 표현이라는 생각에 더 정겨웠다.스타트업 대표들의 창업 동기는 대부분 ‘사

여러 URL을 동시에 스크래핑하면 스크래핑 과정을 가속화할 수 있습니다.

동시 요청에는 합리적인 제한이 있으며, 기본값은 초당 2회입니다. 서버 부하에 대해 걱정하지 않거나, 스크래핑하는 서버를 제어하는 경우라면 `requests_per_second` 매개변수를 변경하여 최대 동시 요청 수를 늘릴 수 있습니다. 이 방법은 스크래핑 속도를 높일 수 있지만, 서버로부터 차단될 수 있으므로 주의해야 합니다.

In [17]:
# jupyter notebook 에서만 실행(asyncio)
import nest_asyncio

nest_asyncio.apply()

In [21]:
# 초당 요청 수 설정
loader.requests_per_second = 1

# 비동기 로드
docs = loader.aload()

Fetching pages: 100%|##########| 2/2 [00:00<00:00,  7.68it/s]


In [22]:
# 결과 출력
docs

[Document(metadata={'source': 'https://n.news.naver.com/article/437/0000378416'}, page_content="\n출산 직원에게 '1억원' 쏜다…회사의 파격적 저출생 정책\n\n\n[앵커]올해 아이 낳을 계획이 있는 가족이라면 솔깃할 소식입니다. 정부가 저출생 대책으로 매달 주는 부모 급여, 0세 아이는 100만원으로 올렸습니다. 여기에 첫만남이용권, 아동수당까지 더하면 아이 돌까지 1년 동안 1520만원을 받습니다. 지자체도 경쟁하듯 지원에 나섰습니다. 인천시는 새로 태어난 아기, 18살될 때까지 1억원을 주겠다. 광주시도 17살될 때까지 7400만원 주겠다고 했습니다. 선거 때면 나타나서 아이 낳으면 현금 주겠다고 밝힌 사람이 있었죠. 과거에는 표만 노린 '황당 공약'이라는 비판이 따라다녔습니다. 그런데 지금은 출산율이 이보다 더 나쁠 수 없다보니, 이런 현금성 지원을 진지하게 정책화 하는 상황까지 온 겁니다. 게다가 기업들도 뛰어들고 있습니다. 이번에는 출산한 직원에게 단번에 1억원을 주겠다는 회사까지 나타났습니다.이상화 기자가 취재했습니다.[기자]한 그룹사가 오늘 파격적인 저출생 정책을 내놨습니다.2021년 이후 태어난 직원 자녀에 1억원씩, 총 70억원을 지원하고 앞으로도 이 정책을 이어가기로 했습니다.해당 기간에 연년생과 쌍둥이 자녀가 있으면 총 2억원을 받게 됩니다.[오현석/부영그룹 직원 : 아이 키우는 데 금전적으로 많이 힘든 세상이잖아요. 교육이나 생활하는 데 큰 도움이 될 거라 생각합니다.]만약 셋째까지 낳는 경우엔 국민주택을 제공하겠다는 뜻도 밝혔습니다.[이중근/부영그룹 회장 : 3년 이내에 세 아이를 갖는 분이 나올 것이고 따라서 주택을 제공할 수 있는 계기가 될 것으로 생각하고.][조용현/부영그룹 직원 : 와이프가 셋째도 갖고 싶어 했는데 경제적 부담 때문에 부정적이었거든요. (이제) 긍정적으로 생각할 수 있을 것 같습니다.]오늘 행사에서는, 회사가 제공하는 출산장려금은 받는 

## 프록시 사용

IP 차단을 우회하기 위해 때때로 프록시를 사용할 필요가 있을 수 있습니다. 

프록시를 사용하려면 로더(및 그 아래의 `requests`)에 프록시 딕셔너리를 전달할 수 있습니다.

In [ ]:
loader = WebBaseLoader(
    "https://www.google.com/search?q=parrots",
    proxies={
        "http": "http://{username}:{password}:@proxy.service.com:6666/",
        "https": "https://{username}:{password}:@proxy.service.com:6666/",
    },
    # 웹 기반 로더 초기화
    # 프록시 설정
)

# 문서 로드
docs = loader.load()